In [ ]:
import requests
import datetime
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

In [27]:
def fetch_finnhub_news(symbol="AVGO", days=30, target_date=None):
    """
    Fetch company news for the given symbol from Finnhub over the past `days` days relative to `target_date`.
    
    Parameters:
      - symbol: Stock ticker symbol (default "AVGO" for Broadcom)
      - days: Number of days in the past to fetch news (default is 30)
      - target_date: The reference date (as a datetime.date object or ISO format string). 
                     If None, uses today's date.
    
    Returns:
      A list of news articles (each is a dictionary with keys like 'headline', 'datetime', etc.)
    """
    # Allow target_date to be a string; if so, convert it to a date object.
    if target_date is None:
        target_date = datetime.date.today()
    elif isinstance(target_date, str):
        try:
            # fromisoformat supports "YYYY-MM-DD" (and extended formats)
            target_date = datetime.datetime.fromisoformat(target_date).date()
        except Exception as e:
            print(f"Error parsing target_date string: {e}. Using today's date instead.")
            target_date = datetime.date.today()
    
    start_date = (target_date - datetime.timedelta(days=days)).strftime("%Y-%m-%d")
    end_date = target_date.strftime("%Y-%m-%d")
    
    url = f"https://finnhub.io/api/v1/company-news?symbol={symbol}&from={start_date}&to={end_date}&token={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        news_data = response.json()
        return news_data
    else:
        print(f"Error fetching news: {response.status_code} - {response.text}")
        return []

In [28]:
# -----------------------------
# FinBERT Sentiment Analysis
# -----------------------------

# Load FinBERT model and tokenizer from HuggingFace
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
finbert_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [29]:
def get_sentiment_score(text):
    """
    Uses FinBERT to compute a sentiment score for the given text.
    Score = (Positive probability - Negative probability), range ~[-1, 1].
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = finbert_model(**inputs)
    scores = torch.softmax(outputs.logits, dim=1)
    # Assuming label order: 0 = negative, 1 = neutral, 2 = positive
    pos_prob = scores[0][2].item()
    neg_prob = scores[0][0].item()
    sentiment = pos_prob - neg_prob
    return sentiment

In [33]:
def compute_sentiment_for_date(target_date_str, symbol="AVGO", decay_rate=0.1):
    """
    Callable parent function that:
      - Accepts a target date string ("YYYY-MM-DD")
      - Fetches news articles for the past 30 days relative to that date
      - Computes FinBERT sentiment scores for each article
      - Applies exponential decay weighting (more recent news gets higher weight)
      - Returns the aggregated weighted sentiment score as an integer.
    
    Parameters:
      - target_date_str: Date for analysis in "YYYY-MM-DD" format.
      - symbol: Stock ticker symbol (default "AVGO")
      - decay_rate: Exponential decay factor (default 0.1)
    
    Returns:
      - Aggregated weighted sentiment score (integer)
    """
    # Convert target_date_str to a datetime.date object
    target_date = datetime.datetime.strptime(target_date_str, "%Y-%m-%d").date()
    
    # Fetch news articles for the past 30 days relative to target_date
    news_articles = fetch_finnhub_news(symbol=symbol, days=30, target_date=target_date)
    
    if not news_articles:
        print("No news articles found.")
        return 0  # Return neutral sentiment if no news found

    # Use target_date (set at midnight) as the reference for weighting
    reference_datetime = datetime.datetime.combine(target_date, datetime.datetime.min.time())
    
    weighted_scores = []
    total_weight = 0.0
    
    for article in news_articles:
        text = article.get('headline', '')
        if not text:
            continue
        
        sentiment = get_sentiment_score(text)
        article_date = datetime.datetime.fromtimestamp(article['datetime'])
        age_days = (reference_datetime - article_date).days
        weight = np.exp(-decay_rate * age_days)
        
        weighted_scores.append(sentiment * weight)
        total_weight += weight
    
    if total_weight > 0:
        aggregated_sentiment = sum(weighted_scores) / total_weight
    else:
        aggregated_sentiment = 0.0

    # Return the aggregated sentiment score as an integer
    return aggregated_sentiment

In [35]:
sentiment_score = compute_sentiment_for_date("2025-03-01")
print(f"Weighted Aggregated Sentiment Score for 2025-03-01: {sentiment_score}")

Weighted Aggregated Sentiment Score for 2025-03-01: 0.39475827822263787
